In [4]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [5]:
cwl_dir = '/home/estorrs/pecgs-pipeline/cwl/msisensor'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

looking at another template for reference

In [2]:
t = yaml.safe_load(open('/home/estorrs/TinJasmine/cwl/template.TinJasmine.yaml'))
t

{'sample_barcode': 'a_string',
 'centromere': {'class': 'File', 'path': 'a/file/path'},
 'chrlist': {'class': 'File', 'path': 'a/file/path'},
 'Canonical_BED': {'class': 'File', 'path': 'a/file/path'},
 'assembly': 'a_string',
 'vep_cache_version': 'a_string',
 'vep_cache_gz': {'class': 'File', 'path': 'a/file/path'},
 'ROI_BED': {'class': 'File', 'path': 'a/file/path'},
 'varscan_filter_config': {'class': 'File', 'path': 'a/file/path'},
 'pindel_filter_config': {'class': 'File', 'path': 'a/file/path'},
 'pindel_config_template': {'class': 'File', 'path': 'a/file/path'},
 'gatk_filter_config': {'class': 'File', 'path': 'a/file/path'},
 'reference': {'class': 'File', 'path': 'a/file/path'},
 'bam': {'class': 'File', 'path': 'a/file/path'}}

test invocation

msisensor msi -d /storage1/fs1/dinglab/Active/Projects/PECGS/ref_genome/hg38.microsatellite -n /storage1/fs1/dinglab/Active/Projects/austins2/HTAN-BRCA/BAM/2021-10-12/HT268B1-Jm1D1_1.N/HT268B1-Jm1D1_1.N.human.remDup.bam -t /storage1/fs1/dinglab/Active/Projects/austins2/HTAN-BRCA/BAM/2021-10-12/HT268B1-Th1H3Y1D1_1.T/HT268B1-Th1H3Y1D1_1.T.human.remDup.bam -o /storage1/fs1/dinglab/Active/Projects/PECGS/MSI_hg38/WXS_test/Mohammad_test/HT268B1 -l 1 -q 1 -b 16

tool options

-d   |string|   homopolymer and microsates file
    
-n   |string|   normal bam file
    
-t   |string|   tumor  bam file
    
-o   |string|   output distribution file

-e   |string|   bed file, optional
    
-f   |double|   FDR threshold for somatic sites detection, default=0.05
    
-c   |int|      coverage threshold for msi analysis, WXS: 20; WGS: 15, default=20
    
-z   |int|      coverage normalization for paired tumor and normal data, 0: no; 1: yes, default=0
    
-r   |string|   choose one region, format: 1:10000000-20000000
    
-l   |int|      minimal homopolymer size, default=5
    
-p   |int|      minimal homopolymer size for distribution analysis, default=10
    
-m   |int|      maximal homopolymer size for distribution analysis, default=50
    
-q   |int|      minimal microsates size, default=3
    
-s   |int|      minimal microsates size for distribution analysis, default=5
    
-w   |int|      maximal microstaes size for distribution analysis, default=40
    
-u   |int|      span size around window for extracting reads, default=500
    
-b   |int|      threads number for parallel computing, default=1
    
-x   |int|      output homopolymer only, 0: no; 1: yes, default=0
    
-y   |int|      output microsatellite only, 0: no; 1: yes, default=0

-h   help

In [3]:
template = {
    'microsatellite': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'normal_bam': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'tumor_bam': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'minimal_homopolymer_size': 'a_int',
    'minimal_microsatellite_size': 'a_int',
    'threads': 'a_int'
}

In [6]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.msisensor.yaml'), 'w'))

make cwl file

looking at another for reference

In [7]:
t = yaml.safe_load(open('/home/estorrs/TinJasmine/cwl/stage_bam.cwl'))
t

{'class': 'CommandLineTool',
 'cwlVersion': 'v1.0',
 '$namespaces': {'sbg': 'https://www.sevenbridges.com/'},
 'id': 'stage_bam',
 'baseCommand': ['bash', '/BICSEQ2/src/stage_file.sh'],
 'inputs': [{'id': 'BAM',
   'type': 'File',
   'inputBinding': {'position': 99},
   'secondaryFiles': ['.bai']}],
 'outputs': [{'id': 'output',
   'type': 'File',
   'outputBinding': {'glob': 'staged_data.bam'},
   'secondaryFiles': ['.bai']}],
 'label': 'stage_bam',
 'arguments': [{'position': 0, 'prefix': '-s', 'valueFrom': '.bai'},
  {'position': 0, 'prefix': '-p', 'valueFrom': 'hard'},
  {'position': 0, 'prefix': '-o', 'valueFrom': 'staged_data.bam'}],
 'requirements': [{'class': 'DockerRequirement',
   'dockerPull': 'mwyczalkowski/bicseq2:20210625'},
  {'class': 'ResourceRequirement', 'ramMin': 8000}]}

In [8]:
t = yaml.safe_load(open('/home/estorrs/TinJasmine/cwl/stage_bam.template.yaml'))
t

{'BAM': {'class': 'File', 'path': 'a/file/path'}}

In [13]:
cwl = {
    'class': 'CommandLineTool',
    'cwlVersion': 'v1.0',
    '$namespaces': {'sbg': 'https://www.sevenbridges.com/'},
    'id': 'msisensor',
    'baseCommand': ['msisensor', 'msi'],
    'inputs': [
        {
            'id': 'microsatellite',
            'type': 'File',
            'inputBinding': {
                'prefix': '-d',
                'position': '0'
            }
        },
        {
            'id': 'normal_bam',
            'type': 'File',
            'inputBinding': {
                'prefix': '-n',
                'position': '0'
            }
        },
        {
            'id': 'tumor_bam',
            'type': 'File',
            'inputBinding': {
                'prefix': '-t',
                'position': '0'
            }
        },
        {
            'id': 'minimal_homopolymer_size',
            'type': 'string?',
            'inputBinding': {
                'prefix': '-l',
                'position': '0'
            }
        },
        {
            'id': 'minimal_microsatellite_size',
            'type': 'string?',
            'inputBinding': {
                'prefix': '-q',
                'position': '0'
            }
        },
        {
            'id': 'threads',
            'type': 'string?',
            'inputBinding': {
                'prefix': '-b',
                'position': '0'
            }
        },
    ],
    'arguments': [
        {
            'position': 0,
            'prefix': '-o',
            'valueFrom': 'output'
        }
    ],
    'outputs': [
        {
            'id': 'output_summary',
            'type': 'File',
            'outputBinding': {'glob': 'output'},     
        },
        {
            'id': 'output_dis',
            'type': 'File',
            'outputBinding': {'glob': 'output_dis'},     
        },
        {
            'id': 'output_germline',
            'type': 'File',
            'outputBinding': {'glob': 'output_germline'},     
        },
        {
            'id': 'output_somatic',
            'type': 'File',
            'outputBinding': {'glob': 'output_somatic'},     
        }
    ],
    'label': 'msisensor',
    'requirements': [
        {
            'class': 'DockerRequirement',
            'dockerPull': 'informationsea/msisensor:0.6'
        },
        {
            'class': 'ResourceRequirement',
            'ramMin': 20000
        }
    ]
}

In [14]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'msisensor.cwl'), 'w'))

In [15]:
t = yaml.safe_load(open('/home/estorrs/TinJasmine/cwl/TinJasmine.cwl'))
t

{'class': 'Workflow',
 'cwlVersion': 'v1.0',
 'id': 'tin_jasmine',
 'label': 'TinJasmine',
 'inputs': [{'id': 'bam', 'type': 'File'},
  {'id': 'reference', 'type': 'File'},
  {'id': 'gatk_filter_config', 'type': 'File'},
  {'id': 'pindel_config_template', 'type': 'File'},
  {'id': 'pindel_filter_config', 'type': 'File'},
  {'id': 'varscan_filter_config', 'type': 'File'},
  {'id': 'ROI_BED', 'type': 'File'},
  {'id': 'vep_cache_gz', 'type': 'File?'},
  {'id': 'vep_cache_version', 'type': 'string?'},
  {'id': 'assembly', 'type': 'string?'},
  {'id': 'Canonical_BED', 'type': 'File'},
  {'id': 'chrlist',
   'type': 'File?',
   'doc': 'List of chromsome regions common to all callers'},
  {'id': 'centromere', 'type': 'File?'},
  {'id': 'sample_barcode', 'type': 'string?'}],
 'outputs': [{'id': 'clean_VCF',
   'outputSource': 'canonical_filter/output',
   'type': 'File'},
  {'id': 'allCall_VCF',
   'outputSource': 'vep_annotate/output_dat',
   'type': 'File'},
  {'id': 'clean_MAF', 'outputSou

In [21]:
cwl = {
    'class': 'Workflow',
    'cwlVersion': 'v1.0',
    'id': 'msisensor_workflow',
    'label': 'msisensor_workflow',
    'inputs': [
        {
            'id': 'microsatellite',
            'type': 'File'
        },
        {
            'id': 'normal_bam',
            'type': 'File'
        },
        {
            'id': 'tumor_bam',
            'type': 'File'
        },
        {
            'id': 'minimal_homopolymer_size',
            'type': 'string?'
        },
        {
            'id': 'minimal_microsatellite_size',
            'type': 'string?'
        },
        {
            'id': 'threads',
            'type': 'string?'
        },
    ],
    'outputs': [
        {
            'id': 'output_summary',
            'type': 'File',
            'outputSource': 'msisensor/output_summary',     
        },
        {
            'id': 'output_dis',
            'type': 'File',
            'outputSource': 'msisensor/output_dis',   
        },
        {
            'id': 'output_germline',
            'type': 'File',
            'outputSource': 'msisensor/output_germline',    
        },
        {
            'id': 'output_somatic',
            'type': 'File',
            'outputSource': 'msisensor/output_somatic',
        }
    ],
    'steps': [
        {
            'id': 'msisensor',
            'in': [
                {
                    'id': 'microsatellite',
                    'source': 'microsatellite'
                },
                {
                    'id': 'minimal_homopolymer_size',
                    'source': 'minimal_homopolymer_size'
                },
                {
                    'id': 'minimal_microsatellite_size',
                    'source': 'minimal_microsatellite_size'
                },
                {
                    'id': 'threads',
                    'source': 'threads'
                },
                {
                    'id': 'normal_bam',
                    'source': 'stage_normal_bam/output'
                },
                {
                    'id': 'tumor_bam',
                    'source': 'stage_tumor_bam/output'
                }
            ],
            'out': [
                {
                    'id': 'output_summary'
                },
                {
                    'id': 'output_dis'
                },
                {
                    'id': 'output_germline'
                },
                {
                    'id': 'output_somatic'
                },
            ],
            'run': './msisensor.cwl',
            'label': 'msisensor'
        },
        {
            'id': 'stage_normal_bam',
            'in': [{'id': 'BAM', 'source': 'normal_bam'}],
            'out': [{'id': 'output'}],
            'run': '../stage_bam/stage_bam.cwl',
            'label': 'stage_normal_bam'
        },
        {
            'id': 'stage_tumor_bam',
            'in': [{'id': 'BAM', 'source': 'tumor_bam'}],
            'out': [{'id': 'output'}],
            'run': '../stage_bam/stage_bam.cwl',
            'label': 'stage_tumor_bam'
        },
    ],
    'requirements': [
        {
            'class': 'ScatterFeatureRequirement'
        }
    ]
}

In [22]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'msisensor_workflow.cwl'), 'w'))